In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import os
from datetime import datetime

# =============================================================================
# CONFIGURACIÓN Y CONSTANTES
# =============================================================================

RUTA_EXCEL = r"C:\Users\Jorge Vasquez\ReporteProduccion\libro2.xlsx"
RUTA_GUARDADO = r"C:\Users\Jorge Vasquez\ReporteProduccion"
RUTA_TEMPLATE = os.path.join(RUTA_GUARDADO, "index.html")
RUTA_HTML_FINAL = os.path.join(RUTA_GUARDADO, "index.html")

# CELDAS DE FECHA Y HORA - FÁCILMENTE CONFIGURABLES
CELDA_FECHA = "A145"
CELDA_HORA = "B145"

NOMBRES_TABLAS = {
    1: "1. Vista Principal",
    2: "2. Vista por Tipificación", 
    3: "3. Vista por Horas",
    4: "4. Vista PDP's por Vencer",
    5: "5. Cantidad de PDP's",
    6: "6. Vista de VLL"
}

COLABORADORES_CONFIG = [
    {"id": "JORDAN", "nombre": "JORDAN - S3 - S4", "celda_inicio": "A6"},
    {"id": "LEONOR", "nombre": "LEONOR - S3 - S4", "celda_inicio": "A30"},
    {"id": "JORGE", "nombre": "JORGE - S2", "celda_inicio": "A54"},
    {"id": "SANDY", "nombre": "SANDY - S1", "celda_inicio": "A78"},
    {"id": "MELINA", "nombre": "MELINA - S1", "celda_inicio": "A102"},
    {"id": "KENNETH", "nombre": "KENNETH - S0", "celda_inicio": "A126"},
]

# =============================================================================
# FUNCIONES DE EXTRACCIÓN DE DATOS
# =============================================================================

def encontrar_rango_datos(ws, celda_inicio_str):
    """Encuentra el rango de datos desde una celda inicial hasta encontrar celdas vacías"""
    try:
        # Convertir coordenada de celda a números
        from openpyxl.utils import coordinate_to_tuple
        fila_inicio, col_inicio = coordinate_to_tuple(celda_inicio_str)
        
        # Encontrar última fila con datos
        fila_actual = fila_inicio
        while ws.cell(row=fila_actual, column=col_inicio).value is not None:
            fila_actual += 1
        ultima_fila = fila_actual - 1
        
        # Encontrar última columna con datos
        col_actual = col_inicio
        while ws.cell(row=fila_inicio, column=col_actual).value is not None:
            col_actual += 1
        ultima_col = col_actual - 1
        
        return fila_inicio, ultima_fila, col_inicio, ultima_col
    except Exception as e:
        print(f"Error en encontrar_rango_datos para {celda_inicio_str}: {e}")
        return fila_inicio, fila_inicio, col_inicio, col_inicio

def obtener_datos_tabla(ws, celda_inicio_str):
    """Obtiene los datos de una tabla desde una celda inicial"""
    try:
        fila_inicio, ultima_fila, col_inicio, ultima_col = encontrar_rango_datos(ws, celda_inicio_str)
        
        # Si no hay datos, retornar lista vacía
        if fila_inicio > ultima_fila or col_inicio > ultima_col:
            return []
        
        datos = []
        for fila in range(fila_inicio, ultima_fila + 1):
            fila_datos = []
            for col in range(col_inicio, ultima_col + 1):
                valor = ws.cell(row=fila, column=col).value
                # Formatear fechas para mostrar solo la fecha
                if isinstance(valor, datetime):
                    valor = valor.strftime('%d/%m/%Y')
                # Formatear específicamente la columna F (columna número 6)
                elif col == 6:  # Columna F absoluta (F = columna 6)
                    try:
                        if isinstance(valor, (int, float)):
                            valor = f"{valor:.1%}"  # Formato 95.0%
                    except:
                        pass
                fila_datos.append(valor if valor is not None else "")
            datos.append(fila_datos)
        
        return datos
    except Exception as e:
        print(f"Error al obtener datos de {celda_inicio_str}: {e}")
        return []

def obtener_fecha_hora_desde_excel(ws):
    """Obtiene la fecha y hora desde las celdas específicas del Excel"""
    try:
        # Obtener fecha y hora desde las celdas configuradas en CONSTANTES
        fecha_celda = ws[CELDA_FECHA].value
        fecha_str = str(fecha_celda) if fecha_celda is not None else pd.Timestamp.now().strftime('%d/%m/%Y')
        
        hora_celda = ws[CELDA_HORA].value
        hora_str = str(hora_celda) if hora_celda is not None else pd.Timestamp.now().strftime('%H:%M')
        
        return fecha_str, hora_str
    except Exception as e:
        print(f"Error al obtener fecha/hora desde Excel: {e}")
        return pd.Timestamp.now().strftime('%d/%m/%Y'), pd.Timestamp.now().strftime('%H:%M')

def extraer_todas_tablas_colaborador(ws, celda_base):
    """Extrae las 6 tablas de un colaborador desde las posiciones predefinidas"""
    # Calcular desplazamientos para las 6 tablas
    desplazamientos = [
        ("", 0),           # Tabla 1: misma posición
        ("I", 0),          # Tabla 2: columna I
        ("AC", 0),         # Tabla 3: columna AC
        ("AP", 0),         # Tabla 4: columna AP
        ("AX", 0),         # Tabla 5: columna AX
        ("BH", 0)          # Tabla 6: columna BH
    ]
    
    tablas = []
    for col_desplazamiento, fila_desplazamiento in desplazamientos:
        # Extraer número de fila de la celda base (ej: "A6" -> 6)
        fila_base = int(''.join(filter(str.isdigit, celda_base)))
        fila_nueva = fila_base + fila_desplazamiento
        
        # Construir nueva celda
        col_base = ''.join(filter(str.isalpha, celda_base))
        if col_desplazamiento:
            celda_nueva = f"{col_desplazamiento}{fila_nueva}"
        else:
            celda_nueva = f"{col_base}{fila_nueva}"
        
        print(f"   📊 Extrayendo tabla desde: {celda_nueva}")
        tabla = obtener_datos_tabla(ws, celda_nueva)
        tablas.append(tabla)
    
    return tablas

# =============================================================================
# FUNCIONES DE GENERACIÓN HTML
# =============================================================================

def generar_html_tabla(datos, titulo=""):
    """Genera el HTML para una tabla individual con botón desplegable"""
    if not datos or len(datos) == 0:
        return f'''
            <div class="tabla-item">
                <button class="tabla-boton" onclick="toggleTabla(this)">{titulo}</button>
                <p style="color: #f39c12; padding: 15px; text-align: center;">📭 Tabla sin datos</p>
            </div>
        '''
    
    html = f'''
            <div class="tabla-item">
                <button class="tabla-boton" onclick="toggleTabla(this)">{titulo}</button>
                <table class="tabla-contenido">
    '''
    
    # Encabezados (primera fila)
    if len(datos) > 0:
        html += '<thead><tr>'
        for celda in datos[0]:
            html += f'<th>{celda}</th>'
        html += '</tr></thead>'
    
    # Datos (filas restantes)
    if len(datos) > 1:
        html += '<tbody>'
        for fila in datos[1:]:
            html += '<tr>'
            for celda in fila:
                html += f'<td>{celda}</td>'
            html += '</tr>'
        html += '</tbody>'
    
    html += '''
                </table>
            </div>
    '''
    
    return html

def generar_seccion_colaborador(colaborador, tablas):
    """Genera la sección HTML completa para un colaborador"""
    seccion_html = f'''
        <div class="grupo" id="{colaborador['id']}">
            <div class="grupo-titulo">{colaborador['nombre']}</div>
            <div class="tablas-container">
    '''
    
    # Agregar las 6 tablas del colaborador
    for i in range(6):
        seccion_html += generar_html_tabla(tablas[i], NOMBRES_TABLAS[i+1])
    
    seccion_html += '''
            </div>
        </div>
    '''
    
    return seccion_html

def cargar_template():
    """Carga el template HTML base"""
    try:
        with open(RUTA_TEMPLATE, 'r', encoding='utf-8') as f:
            return f.read()
    except FileNotFoundError:
        print(f"❌ Error: No se encontró el archivo template en {RUTA_TEMPLATE}")
        print("💡 Asegúrate de haber creado el archivo template.html")
        return None

def guardar_html_final(html_content):
    """Guarda el HTML final generado"""
    try:
        with open(RUTA_HTML_FINAL, 'w', encoding='utf-8') as f:
            f.write(html_content)
        print(f"✅ Archivo HTML generado exitosamente en: {RUTA_HTML_FINAL}")
        return True
    except Exception as e:
        print(f"❌ Error al guardar el archivo HTML: {e}")
        return False

# =============================================================================
# FUNCIÓN PRINCIPAL
# =============================================================================

def main():
    """Función principal del script"""
    print("🚀 Iniciando generación de reporte...")
    
    # Verificar que el template existe
    template = cargar_template()
    if template is None:
        return
    
    # Crear directorio si no existe
    os.makedirs(RUTA_GUARDADO, exist_ok=True)
    
    # Cargar y procesar el archivo Excel
    try:
        wb = load_workbook(RUTA_EXCEL)
        ws = wb.active
        
        print("📊 Extrayendo datos de todos los colaboradores...")
        
        # Obtener fecha y hora desde el Excel
        fecha_reporte, hora_reporte = obtener_fecha_hora_desde_excel(ws)
        print(f"📅 Fecha obtenida: {fecha_reporte}")
        print(f"⏰ Hora obtenida: {hora_reporte}")
        
        # Extraer datos de todos los colaboradores
        datos_colaboradores = {}
        for colaborador in COLABORADORES_CONFIG:
            print(f"👤 Procesando: {colaborador['nombre']}")
            tablas = extraer_todas_tablas_colaborador(ws, colaborador['celda_inicio'])
            datos_colaboradores[colaborador['id']] = tablas
            
            # Mostrar estadísticas de tablas
            tablas_con_datos = sum(1 for tabla in tablas if len(tabla) > 0)
            print(f"   ✅ {tablas_con_datos}/6 tablas con datos")
        
    except Exception as e:
        print(f"❌ Error al cargar el archivo Excel: {e}")
        return
    
    # Generar contenido dinámico
    print("\n🛠️ Generando HTML...")
    
    # Generar navegación
    navegacion_html = ""
    for colaborador in COLABORADORES_CONFIG:
        navegacion_html += f'<a href="#{colaborador["id"]}">▶ {colaborador["nombre"]}</a>\n'
    
    # Generar contenido principal
    contenido_principal_html = ""
    for colaborador in COLABORADORES_CONFIG:
        tablas = datos_colaboradores[colaborador['id']]
        contenido_principal_html += generar_seccion_colaborador(colaborador, tablas)
    
    # Reemplazar placeholders en el template
    html_final = template
    html_final = html_final.replace('{fecha_reporte}', fecha_reporte)
    html_final = html_final.replace('{hora_reporte}', hora_reporte)
    html_final = html_final.replace('{navegacion}', navegacion_html)
    html_final = html_final.replace('{contenido_principal}', contenido_principal_html)
    html_final = html_final.replace('{fecha_actualizacion}', pd.Timestamp.now().strftime('%d/%m/%Y %H:%M'))
    
    # Guardar el HTML final
    if guardar_html_final(html_final):
        # Mostrar resumen final
        print("\n📈 RESUMEN FINAL:")
        print(f"• Total colaboradores procesados: {len(COLABORADORES_CONFIG)}")
        print(f"• Total tablas generadas: {len(COLABORADORES_CONFIG) * 6}")
        print(f"• Fecha del reporte: {fecha_reporte}")
        print(f"• Hora del corte: {hora_reporte}")
        print(f"• Archivo generado: {RUTA_HTML_FINAL}")
        print("\n🎯 ¡Proceso completado exitosamente!")

# =============================================================================
# EJECUCIÓN
# =============================================================================

if __name__ == "__main__":
    main()

🚀 Iniciando generación de reporte...
📊 Extrayendo datos de todos los colaboradores...
📅 Fecha obtenida: 5/11/2025
⏰ Hora obtenida: 10AM
👤 Procesando: JORDAN - S3 - S4
   📊 Extrayendo tabla desde: A6
   📊 Extrayendo tabla desde: I6
   📊 Extrayendo tabla desde: AC6
   📊 Extrayendo tabla desde: AP6
   📊 Extrayendo tabla desde: AX6
   📊 Extrayendo tabla desde: BH6
   ✅ 6/6 tablas con datos
👤 Procesando: LEONOR - S3 - S4
   📊 Extrayendo tabla desde: A30
   📊 Extrayendo tabla desde: I30
   📊 Extrayendo tabla desde: AC30
   📊 Extrayendo tabla desde: AP30
   📊 Extrayendo tabla desde: AX30
   📊 Extrayendo tabla desde: BH30
   ✅ 6/6 tablas con datos
👤 Procesando: JORGE - S2
   📊 Extrayendo tabla desde: A54
   📊 Extrayendo tabla desde: I54
   📊 Extrayendo tabla desde: AC54
   📊 Extrayendo tabla desde: AP54
   📊 Extrayendo tabla desde: AX54
   📊 Extrayendo tabla desde: BH54
   ✅ 6/6 tablas con datos
👤 Procesando: SANDY - S1
   📊 Extrayendo tabla desde: A78
   📊 Extrayendo tabla desde: I78
   📊 Ext

In [2]:
import subprocess
import os

# Ruta donde está tu repositorio
repo_path = r"C:\Users\Jorge Vasquez\ReporteProduccion"

try:
    # Navegar al repositorio
    os.chdir(repo_path)
    
    # Comandos Git para actualizar
    subprocess.run(["git", "add", "index.html"], check=True)
    subprocess.run(["git", "commit", "-m", f"Actualización automática - {pd.Timestamp.now().strftime('%d/%m/%Y %H:%M')}"], check=True)
    subprocess.run(["git", "push"], check=True)
    
    print("✅ GitHub actualizado automáticamente")
    
except Exception as e:
    print(f"❌ Error al actualizar GitHub: {e}")

❌ Error al actualizar GitHub: Command '['git', 'commit', '-m', 'Actualización automática - 05/11/2025 14:46']' returned non-zero exit status 1.
